In [8]:
import pandas as pd
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

In [11]:
def preprocess_(path,  slice, column_, encoding=None, header='infer'):
    
    df = pd.read_csv(path, encoding=encoding, header=header)
    
    if slice:
        df = df[slice]

    df.columns = column_

    if (type(df.iloc[:,1][1])==str):
        df.iloc[:,1] = df.iloc[:,1].apply(lambda x: float(x.replace(',', '')))
        
    df['Date'] = df['Date'].str.replace('[^0-9]', '')
    df['Date'] = df['Date'].apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
    df['Date'] = pd.to_datetime(df['Date'])

    df.sort_values('Date', inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], axis=1, inplace=True)

    return df

In [12]:
# 데이터 불러오기
Gold = pd.read_csv('./csv/Gold.csv')
REMX = pd.read_csv('./csv/REMX.csv')
AAPL = pd.read_csv('./csv/AAPL.csv')
TSMC = pd.read_csv('./csv/TSMC.csv')
ExchangeRate = pd.read_csv('./csv/ExchangeRate.csv')

In [14]:
# 병합
from functools import reduce
dfs = [Gold, REMX, AAPL, TSMC, ExchangeRate]
union = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'),dfs)
union

,Date,Gold,REMX,AAPL,TSMC,USD/KRW
0,2020-05-01,1728.6,29.93,72.27,51.10,1224.89
1,2020-05-04,1748.5,29.89,73.29,52.15,1224.14
2,2020-05-05,1751.3,31.01,74.39,51.71,1222.47
3,2020-05-06,1729.2,31.07,75.16,52.13,1228.55
4,2020-05-07,1768.6,31.08,75.93,52.33,1219.61
...,...,...,...,...,...,...
393,2022-03-25,1959.8,116.80,174.72,106.73,1225.08
394,2022-03-28,1944.7,114.81,175.60,106.33,1223.65
395,2022-03-29,1918.0,117.54,178.96,109.25,1207.35
396,2022-03-30,1939.0,117.90,177.77,106.73,1210.31


In [15]:
# union.to_csv('./csv/merge.csv', index=None)